In [90]:

import pandas as pds
import math
import numpy as npy  
import seaborn as seabrn
import warnings
import random
from sklearn.metrics import confusion_matrix
from math import sqrt
from math import exp
from math import pi
import csv

my_training_dataset_data =pds.read_csv(r"/Users/vussh/Downloads/optdigits_69_N200.tra",header=None)
my_training_dataset_data






     
    




,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,64
0,0,0,5,14,4,0,0,0,0,0,...,0,0,0,4,12,14,7,0,0,6
1,0,0,0,10,12,0,0,0,0,0,...,0,0,0,1,11,14,12,1,0,6
2,0,0,2,14,10,0,0,0,0,1,...,0,0,0,1,12,14,12,4,0,6
3,0,0,2,13,5,0,0,0,0,0,...,0,0,0,4,15,12,9,1,0,6
4,0,0,2,13,6,0,0,0,0,0,...,0,0,0,2,12,14,12,4,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0,0,8,13,10,5,0,0,0,...,0,0,0,0,14,15,2,0,0,9
196,0,0,0,3,10,13,12,4,0,0,...,0,0,0,0,1,15,1,0,0,9
197,0,0,0,4,10,13,12,0,0,0,...,0,0,0,0,2,11,0,0,0,9
198,0,0,6,14,16,9,0,0,0,5,...,0,0,0,8,14,16,13,1,0,9


In [91]:
training_dataset_summary = my_training_dataset_data.to_numpy()
misclassified_value = 0

def classify_by_class(dataset_loc):
    classified = dict()
    for k in range(len(dataset_loc)):
        vector_loc = dataset_loc[k]
        class_value_loc = vector_loc[-1]
        if (class_value_loc not in classified):
            classified[class_value_loc] = list()
        classified[class_value_loc].append(vector_loc)
    return classified   
    
predicted_output_value = list()
mean_array_for_6 = list()
mean_array_for_9 = list()
non_zerolist = my_training_dataset_data

def summarize_dataset_data(dataset_loc):
    summaries_result = [(npy.mean(column_val), npy.var(column_val)) for column_val in zip(*dataset_loc)]
    del(summaries_result[-1])
    return summaries_result

In [92]:
summary_for_6_traindata = list(summarize_dataset_data(classify_by_class(training_dataset_summary).get(6)))
summary_for_9_traindata = list(summarize_dataset_data(classify_by_class(training_dataset_summary).get(9)))

In [93]:
actual_output_value = my_training_dataset_data[64]

for column_val in my_training_dataset_data:
    if column_val == 64:
        break
    mean_for_6 = summary_for_6_traindata[column_val][0]
    variance_for_6 = summary_for_6_traindata[column_val][1]
    mean_for_9 = summary_for_9_traindata[column_val][0]
    variance_for_9 = summary_for_9_traindata[column_val][1]
    
    if variance_for_6 == 0 or variance_for_9 == 0:
        non_zerolist.drop([column_val],axis=1,inplace=True)
    else:
        mean_array_for_6.append(mean_for_6)
        mean_array_for_9.append(mean_for_9)

In [94]:
non_zerolist=non_zerolist.to_numpy()
dataset_for_6 = list(classify_by_class(non_zerolist).get(6))
dataset_for_9 = list(classify_by_class(non_zerolist).get(9))
dataset_for_6 = npy.delete(dataset_for_6, -1, axis=1)
dataset_for_9 = npy.delete(dataset_for_9, -1, axis=1)

In [95]:
cov_matrix_for_6 = npy.cov(dataset_for_6,rowvar=False,bias=True)  
cov_matrix_for_9 = npy.cov(dataset_for_9,rowvar=False,bias=True)   
deter_for_6 = npy.linalg.det(cov_matrix_for_6)
deter_for_9 = npy.linalg.det(cov_matrix_for_9)
mean_array_for_6=npy.array(mean_array_for_6)
transpose_for_m6 = npy.transpose(mean_array_for_6)
transpose_for_m9 = npy.transpose(mean_array_for_6)
inv_cov_matrix_for_6 = npy.linalg.inv(cov_matrix_for_6)
inv_cov_matrix_for_9 = npy.linalg.inv(cov_matrix_for_9)
W_6 = (-0.5*inv_cov_matrix_for_6)
W_9 = (-0.5*inv_cov_matrix_for_9)
w_6 = (inv_cov_matrix_for_6.dot(mean_array_for_6))
w_9 = (inv_cov_matrix_for_9.dot(mean_array_for_9))
W_60 = (-0.5*(transpose_for_m6.dot(inv_cov_matrix_for_6)).dot(mean_array_for_6))-(0.5*math.log10(abs(deter_for_6))+(math.log10(0.5)))
W_90 = (-0.5*(transpose_for_m9.dot(inv_cov_matrix_for_9)).dot(mean_array_for_9))-(0.5*math.log10(abs(deter_for_9))+(math.log10(0.5)))


In [96]:
for row_value in range(0,len(non_zerolist)): 
    
    x = npy.delete(non_zerolist[row_value], -1)
    
    g_of_6_x =  (((npy.transpose(x)).dot(W_6)).dot(x)) +((npy.transpose(w_6)).dot(x)) + W_60
    g_of_9_x =  (((npy.transpose(x)).dot(W_9)).dot(x)) +((npy.transpose(w_9)).dot(x)) + W_90
        
    if(g_of_6_x > g_of_9_x ):
        prediction_is = 6
        predicted_output_value.append(prediction_is)
    else:
        prediction_is = 9
        predicted_output_value.append(prediction_is)
    
    if (prediction_is!=actual_output_value[row_value]):
        misclassified_value = misclassified_value + 1
    print("The predicted and actualoutput: " , (predicted,actual_output_value[row]))


The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)
The predicted and actualoutput:  (9, 9)


In [97]:
training_error_rate = misclassified_value/361
print("There are ",misclassified_value," misclassified rows")
print("error rate is ",training_error_rate)
confusion_matrix(actual_output_value, predicted_output_value) 

There are  0  misclassified rows
error rate is  0.0


array([[100,   0],
       [  0, 100]], dtype=int64)